### 資料、套件導入

In [1]:
import os
import pandas as pd
import re 

In [2]:
path = '../data/0414/review_dataset.csv'
df = pd.read_csv(path)
df = df[:1272]
df

,reviews,value,comfort,location,cleanliness,service,facilities
0,價格合理 舒適的房間 老闆娘人很好還會自己做早餐給旅客 重點是早餐吃到飽,1.0,1.0,0.0,0.0,1.0,0.0
1,內部房間滿乾淨的，還有大場地，可以讓團體來使用，我覺得很棒,0.0,0.0,0.0,1.0,0.0,1.0
2,隔音還可以，房間不小，但美中不足沒有乾濕分離,0.0,1.0,0.0,0.0,0.0,0.0
3,房子設計的很棒 房間採光很好 大廳挑高氣派 房價合理 台東必住民宿,1.0,1.0,0.0,0.0,0.0,1.0
4,Cp值高 乾淨舒適空間大 樓下有免費吐司和咖啡 老闆回復速度快,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
1267,港式飲茶餐廳口味很棒，環境乾淨，機車汽車都有停車位，位於高鐵附近，挺適合宴客的。,0.0,0.0,1.0,1.0,1.0,1.0
1268,場地氣派、丁香魚酥脆、服務親切 蠟味蘿蔔糕份量多一些更好、牛肉粥好吃,0.0,0.0,0.0,0.0,1.0,0.0
1269,交通方便，地下室停車場良好，菜色好，空間設計好，真可說是一流的飯店。,0.0,0.0,1.0,0.0,1.0,1.0
1270,地點佳，離逢甲夜市很近，老闆娘很親切服務，房間夠大舒適，浴室也很乾淨,0.0,1.0,1.0,1.0,1.0,0.0


### 移除重複值、空值

In [3]:
#印出重複資料
df[df.duplicated()]

,reviews,value,comfort,location,cleanliness,service,facilities


In [4]:
#移除重複值
print(df.shape)
df = df.drop_duplicates() #移除
print(df.shape)

(1272, 7)
(1272, 7)


In [5]:
#印出空值資料
df[df.isnull().T.any()]

,reviews,value,comfort,location,cleanliness,service,facilities


### 拆分數據集

In [10]:
train_df = df[:int(df.shape[0]*(2/3))]
test_df = df[int(df.shape[0]*(2/3)):]
print("訓練集：",train_df.shape)
print("測試集：",test_df.shape)
test_df = test_df.reset_index(drop=True) #重新index

訓練集： (840, 7)
測試集： (420, 7)


### 資料前處理：移除標點符號、空格

In [12]:
def review_preprocessing(sentence):
    #sentence = '很乾淨，停車方便， 住的那天房價很便宜， 房間和外觀都超可愛\t'
    ## 把標點符號、空格拿掉
    pattern = re.compile(r'[^\w]') #匹配字母數字及下劃線
    sentence = re.sub(pattern, '', sentence)
    return sentence

In [13]:
def review_preprocessing_run(df):
    for i, row in df.iterrows():
        ifor_val = review_preprocessing(row['reviews']) #資料處理
        df.at[i,'reviews'] = ifor_val #更新資料
    return df

In [14]:
train_df = review_preprocessing_run(train_df)
test_df = review_preprocessing_run(test_df)

### 中文分詞+詞性標注

In [15]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
import gzip, bz2
from datetime import datetime, timedelta

In [16]:
POS = True #是否也要一同執行詞性標記
KEEP_PUNCTUATION_POS = False

In [17]:
device = -1 # 0=GPU, -1=CPU
level = 3 #level: 3=most accurate, but model size largest
print("Initializing ws driver", flush=True) #flush 實現動態Loading
ws_driver  = CkipWordSegmenter(device=device, level=level)
if POS:
    print("Done initializing POS driver", flush=True)
    pos_driver = CkipPosTagger(device=device, level=level)
    print("Initializing pos driver", flush=True)

Initializing ws driver
Done initializing POS driver
Initializing pos driver


In [18]:
def ws_pos_review(sentence):
    ws  = ws_driver([sentence])
    pos = pos_driver(ws) 

    # 打包結果
    assert len(ws[0]) == len(pos[0]) #檢查 ws 和 pos是否長度一致
    res = []
    for word_ws, word_pos in zip(ws[0], pos[0]):
        res.append(f"{word_ws}({word_pos})")
    
    return ",".join(res)

In [19]:
def ws_pos_review_run(df):
    for i, row in df.iterrows():
        ws_pos_list = ws_pos_review(row['reviews']) #分詞+詞性標注
        df.at[i,'ws_pos_reviews'] = ws_pos_list #更新資料
        print("目前進度：" + str(i))
    return df

In [20]:
train_df_ws = ws_pos_review_run(train_df)
test_df_ws = ws_pos_review_run(test_df)

Inference: 100%|██████████| 1/1 [00:00<00:00, 14.44it/s]
C:\Users\g3190\AppData\Local\Temp/ipykernel_16708/27497809.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[i,'ws_pos_reviews'] = ws_pos_list #更新資料


目前進度：0


Inference: 100%|██████████| 1/1 [00:00<00:00, 16.65it/s]


目前進度：1


Inference: 100%|██████████| 1/1 [00:00<00:00, 17.97it/s]


...

目前進度：417


Inference: 100%|██████████| 1/1 [00:00<00:00, 16.39it/s]


目前進度：418


Inference: 100%|██████████| 1/1 [00:00<00:00, 13.16it/s]

目前進度：419


In [21]:
train_df_ws

,reviews,value,comfort,location,cleanliness,service,facilities,ws_pos_reviews
0,價格合理舒適的房間老闆娘人很好還會自己做早餐給旅客重點是早餐吃到飽,1.0,1.0,0.0,0.0,1.0,0.0,"價格(Na),合理(VH),舒適(VH),的(DE),房間(Nc),老闆娘(Na),人(Na..."
1,內部房間滿乾淨的還有大場地可以讓團體來使用我覺得很棒,0.0,0.0,0.0,1.0,0.0,1.0,"內部(Ncd),房間(Nc),滿(Dfa),乾淨(VH),的(DE),還(D),有(V_2)..."
2,隔音還可以房間不小但美中不足沒有乾濕分離,0.0,1.0,0.0,0.0,0.0,1.0,"隔音(A),還可以(D),房間(Nc),不(D),小(VH),但(Cbb),美中不足(VH)..."
3,房子設計的很棒房間採光很好大廳挑高氣派房價合理台東必住民宿,1.0,1.0,0.0,0.0,0.0,0.0,"房子(Na),設計(VC),的(DE),很(Dfa),棒(VH),房間(Nc),採光(Na)..."
4,Cp值高乾淨舒適空間大樓下有免費吐司和咖啡老闆回復速度快,1.0,1.0,0.0,0.0,1.0,0.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...,...,...,...,...,...,...
835,老闆體貼又親切又有美味早餐吃超推薦,0.0,0.0,0.0,0.0,1.0,0.0,"老闆(Na),體貼(VJ),又(Caa),親切(VH),又(Caa),有(V_2),美味(V..."
836,平價老闆娘親切但床單似乎不乾淨身體會癢,1.0,0.0,0.0,1.0,1.0,0.0,"平價(Na),老闆娘(Na),親切(VH),但(Cbb),床單(Na),似乎(D),不(D)..."
837,專業型開飲機有小陽台曬一下衣服隔天就會乾老闆很親切很優的民宿推,0.0,0.0,0.0,0.0,1.0,1.0,"專業型(Na),開飲機(Na),有(V_2),小(VH),陽台(Na),曬(VJ),一下(N..."
838,房間舒適床很好睡不在鬧區所以摩托車噪音少一覺好眠,0.0,1.0,1.0,0.0,0.0,0.0,"房間(Nc),舒適(VH),床(Na),很(Dfa),好(VH),睡(VA),不(D),在(..."


In [22]:
test_df_ws

,reviews,value,comfort,location,cleanliness,service,facilities,ws_pos_reviews
0,而且廚房有提供餅乾飲料可以買其實也可以不用去到全家買,0.0,0.0,0.0,0.0,1.0,0.0,"而且(Cbb),廚房(Nc),有(V_2),提供(VD),餅乾(Na),飲料(Na),可以(..."
1,而且離車站客運站都很近有機會會想回住,0.0,0.0,1.0,0.0,0.0,0.0,"而且(Cbb),離(P),車站(Nc),客運站(Nc),都(D),很(Dfa),近(VH),..."
2,離車站好近房間很大隨時訂放都有人接洽雖然不是主題民宿推薦洽公背包客住宿,0.0,1.0,1.0,0.0,1.0,0.0,"離(P),車站(Nc),好(Dfa),近(VH),房間(Nc),很(Dfa),大(VH),隨..."
3,4人房空間超大床埔也大整體乾淨舒適超讚,0.0,1.0,0.0,1.0,0.0,0.0,"4(Neu),人(Na),房(Na),空間(Na),超大(A),床埔(Nc),也(D),大(..."
4,價格公道的背包客民宿老闆也蠻客氣的旁邊還有機車等代步工具可租乘,1.0,0.0,0.0,0.0,1.0,1.0,"價格(Na),公道(VH),的(DE),背包客(Na),民宿(Nc),老闆(Na),也(D)..."
...,...,...,...,...,...,...,...,...
415,房間就可以看到海馬哥非常熱情民宿風格簡約很多ikea的東西,0.0,0.0,1.0,0.0,1.0,1.0,"房間(Nc),就(D),可以(D),看到(VE),海馬哥(Nb),非常(Dfa),熱情(VH..."
416,老闆娘還說如果先問可以借嬰兒小推車我們就不用帶了帶小孩吃烤肉必備,0.0,0.0,0.0,0.0,0.0,1.0,"老闆娘(Na),還(D),說(VE),如果(Cbb),先(D),問(VE),可以(D),借(..."
417,乾淨舒適闆娘和小幫手超熱情推薦在地景點及美食招待的水果好甜好好吃完食,0.0,1.0,0.0,1.0,1.0,0.0,"乾淨(VH),舒適(VH),闆娘(Na),和(Caa),小(VH),幫手(Na),超(Dfa..."
418,老闆娘親切贈予水果點心房間舒適整潔小孩很喜歡,0.0,1.0,0.0,1.0,1.0,0.0,"老闆娘(Na),親切(VH),贈予(VD),水果(Na),點心(Na),房間(Nc),舒適(..."


### 移除停用詞

In [23]:
def remove_stop(df):
    df_2 = df #將分詞、詞性標註的結果存到另一個df裡 
    stopwords_path='../data/stopwords_TW2.txt' #使用哈工大停用詞表
    # 設定停用詞
    print('start read stopwords data.')
    stopwords = []
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        for line in f:
            if len(line)>0:
                stopwords.append(line.strip())

    rawlist = df_2['ws_pos_reviews'].tolist() 

    for idx,element in enumerate(rawlist):
        filter_list = []
        rawlist2 = element.split(",") #轉成 list
        for val in rawlist2: #去比較list裡面每一個元素，是否屬於stopwords
            extract_str = re.search('(.*?)\(', val).group(1) #擷取"("前面的字串
            if(extract_str not in stopwords): # 如果屬於stopwords，就append到新的filter_list
                filter_list.append(val)
        df_2.at[idx,'filtered'] = ",".join(filter_list) #更新df_2資料
    return df_2

In [24]:
train_df = remove_stop(train_df_ws)
test_df = remove_stop(test_df_ws)

start read stopwords data.


C:\Users\g3190\AppData\Local\Temp/ipykernel_16708/4062774324.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.at[idx,'filtered'] = ",".join(filter_list) #更新df_2資料


start read stopwords data.


### 存檔

In [25]:
train_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_train.csv', encoding='utf_8_sig', index=False)
test_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_test.csv', encoding='utf_8_sig', index=False)

### 擷取名詞和動詞、外語、介詞

In [26]:
def getword(sentence):
    #sentence = '價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),很(Dfa),好(VH),還(D),會(D),做(VC),早餐(Na),旅客(Na),重點(Na),早餐(Na),吃到飽(VC)'
    res=[]
    pattern = r"\b[^,]*\([N,V,F,P][^,]*"
    matches = re.finditer(pattern, sentence)
    for matchNum, match in enumerate(matches, start=1):
        res.append(match.group()) #放進list
    #將list轉成string
    filter_str = ','.join(res)
    return filter_str

In [27]:
train_df['filtered_word'] = train_df.apply(lambda x: getword(x['filtered']),axis=1)
test_df['filtered_word'] = test_df.apply(lambda x: getword(x['filtered']),axis=1)

C:\Users\g3190\AppData\Local\Temp/ipykernel_16708/4185028873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['filtered_word'] = train_df.apply(lambda x: getword(x['filtered']),axis=1)


In [29]:
test_df

,reviews,value,comfort,location,cleanliness,service,facilities,ws_pos_reviews,filtered,filtered_word
0,而且廚房有提供餅乾飲料可以買其實也可以不用去到全家買,0.0,0.0,0.0,0.0,1.0,0.0,"而且(Cbb),廚房(Nc),有(V_2),提供(VD),餅乾(Na),飲料(Na),可以(...","廚房(Nc),提供(VD),餅乾(Na),飲料(Na),買(VC),不用(D),全(Neqa...","廚房(Nc),提供(VD),餅乾(Na),飲料(Na),買(VC),全(Neqa),家(Nc..."
1,而且離車站客運站都很近有機會會想回住,0.0,0.0,1.0,0.0,0.0,0.0,"而且(Cbb),離(P),車站(Nc),客運站(Nc),都(D),很(Dfa),近(VH),...","離(P),車站(Nc),客運站(Nc),機會(Na),想(VE),回住(VCL)","離(P),車站(Nc),客運站(Nc),機會(Na),想(VE),回住(VCL)"
2,離車站好近房間很大隨時訂放都有人接洽雖然不是主題民宿推薦洽公背包客住宿,0.0,1.0,1.0,0.0,1.0,0.0,"離(P),車站(Nc),好(Dfa),近(VH),房間(Nc),很(Dfa),大(VH),隨...","離(P),車站(Nc),好(Dfa),房間(Nc),隨時(D),訂放(VC),人(Na),接...","離(P),車站(Nc),房間(Nc),訂放(VC),人(Na),接洽(VC),主題(Na),..."
3,4人房空間超大床埔也大整體乾淨舒適超讚,0.0,1.0,0.0,1.0,0.0,0.0,"4(Neu),人(Na),房(Na),空間(Na),超大(A),床埔(Nc),也(D),大(...","人(Na),房(Na),空間(Na),超大(A),床埔(Nc),整體(Na),乾淨(VH),...","人(Na),房(Na),空間(Na),床埔(Nc),整體(Na),乾淨(VH),舒適(VH)"
4,價格公道的背包客民宿老闆也蠻客氣的旁邊還有機車等代步工具可租乘,1.0,0.0,0.0,0.0,1.0,1.0,"價格(Na),公道(VH),的(DE),背包客(Na),民宿(Nc),老闆(Na),也(D)...","價格(Na),公道(VH),背包客(Na),民宿(Nc),老闆(Na),客氣(VH),旁邊(...","價格(Na),公道(VH),背包客(Na),民宿(Nc),老闆(Na),客氣(VH),旁邊(..."
...,...,...,...,...,...,...,...,...,...,...
415,房間就可以看到海馬哥非常熱情民宿風格簡約很多ikea的東西,0.0,0.0,1.0,0.0,1.0,1.0,"房間(Nc),就(D),可以(D),看到(VE),海馬哥(Nb),非常(Dfa),熱情(VH...","房間(Nc),看到(VE),海馬哥(Nb),熱情(VH),民宿(Nc),風格(Na),簡約(...","房間(Nc),看到(VE),海馬哥(Nb),熱情(VH),民宿(Nc),風格(Na),簡約(..."
416,老闆娘還說如果先問可以借嬰兒小推車我們就不用帶了帶小孩吃烤肉必備,0.0,0.0,0.0,0.0,0.0,1.0,"老闆娘(Na),還(D),說(VE),如果(Cbb),先(D),問(VE),可以(D),借(...","老闆娘(Na),說(VE),先(D),問(VE),嬰兒(Na),小推車(VH),不用(D),...","老闆娘(Na),說(VE),問(VE),嬰兒(Na),小推車(VH),小孩(Na),吃(VC..."
417,乾淨舒適闆娘和小幫手超熱情推薦在地景點及美食招待的水果好甜好好吃完食,0.0,1.0,0.0,1.0,1.0,0.0,"乾淨(VH),舒適(VH),闆娘(Na),和(Caa),小(VH),幫手(Na),超(Dfa...","乾淨(VH),舒適(VH),闆娘(Na),小(VH),幫手(Na),超(Dfa),熱情(VH...","乾淨(VH),舒適(VH),闆娘(Na),小(VH),幫手(Na),熱情(VH),推薦(VC..."
418,老闆娘親切贈予水果點心房間舒適整潔小孩很喜歡,0.0,1.0,0.0,1.0,1.0,0.0,"老闆娘(Na),親切(VH),贈予(VD),水果(Na),點心(Na),房間(Nc),舒適(...","老闆娘(Na),親切(VH),贈予(VD),水果(Na),點心(Na),房間(Nc),舒適(...","老闆娘(Na),親切(VH),贈予(VD),水果(Na),點心(Na),房間(Nc),舒適(..."


In [30]:
train_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_train_n+v+f+p.csv', encoding='utf_8_sig', index=False)
test_df.to_csv('../data/0407/review_data(seg+pos+stopwords)_test_n+v+f+p.csv', encoding='utf_8_sig', index=False)